# Builds GH's 2 year scorecard

In [1]:
# API settings
api_url = '/services/data/v43.0/sobjects'
external_id = 'Name'
object_id = 'ContactToolAssociation__c'

# login parameters to be handled by Papermill
sql_login_params = {"username": "wang2506_ro", "password": "fnVnwcCS7iT45EsA"}
sf_login_params = {
    "grant_type": "password",
    "client_id": "3MVG95jctIhbyCppj0SNJ75IsZ1y8UPGZtSNF4j8FNVXz.De8Lu4jHm3rjRosAtsHy6qjHx3i4S_QbQzvBePG",
    "client_secret": "D1623C6D3607D4FC8004B92C761DFB6C1F70CCD129C5501E357028DFA00F5764",
    "username":"wang2506@purdue.edu",
    "password":"purdue99272844vp8Z6qq0teuPQmirGaaPAt4",   
} 
    #"username": "wang159-4j1v@force.com",
    #"password": "napoleon0eZ3PQpQqE3C3z4wWqEqKGhQ8",

In [2]:
import sys
sys.path.append('/home/users/wang2506/nanohub_salesforce_integ/salesforce')
    
import pandas as pd
import time
import datetime

from DB2SalesforceAPI import DB2SalesforceAPI

## Pull data from SF

In [3]:
##########################################################################################
## 1 - Pull all tool_usage_cluster__c information
##     i) ID
##     ii) Ending_Date__c
##     iii) Starting_Date__c
##     iv) Organization__c
##     v) Estimated_cluster_size__c
##     vi) Estimated_number_of_tools__c
##########################################################################################

In [4]:
db_1 = DB2SalesforceAPI(sf_login_params)

Obtained Salesforce access token ...... True


In [5]:
tool_cluster_df = db_1.query_data('Select ID, Ending_Date__c, Starting_Date__c, Organization__c,Estimated_cluster_size__c, \
                        Estimated_number_of_tools__c from tool_usage_cluster__c')

[Success] Bulk job creation successful. Job ID = 7505w00000TmbqQAAR


{"id":"7505w00000TmbqQAAR","operation":"query","object":"tool_usage_cluster__c","createdById":"0055w00000DM5bOAAT","createdDate":"2021-01-22T17:11:27.000+0000","systemModstamp":"2021-01-22T17:11:27.000+0000","state":"UploadComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","retries":0,"totalProcessingTime":0}


{"id":"7505w00000TmbqQAAR","operation":"query","object":"tool_usage_cluster__c","createdById":"0055w00000DM5bOAAT","createdDate":"2021-01-22T17:11:27.000+0000","systemModstamp":"2021-01-22T17:11:37.000+0000","state":"JobComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":3944,"retries":0,"totalProcessingTime":700}
[Success] Bulk job completed successfully.


In [6]:
display(tool_cluster_df.head(5))

,Ending_Date__c,Estimated_cluster_size__c,Estimated_number_of_tools__c,Id,Organization__c,Starting_Date__c
0,2011-01-30,12.0,5.0,a0w5w000009Q7CuAAK,NaN,2010-09-05
1,2018-11-30,5.0,1.0,a0w5w000009Q7CvAAK,a0r5w00000V42iyAAB,2018-11-29
2,2019-11-28,39.0,3.0,a0w5w000009Q7CwAAK,a0r5w00000V42iyAAB,2019-11-04
3,2017-10-29,44.0,2.0,a0w5w000009Q7CxAAK,a0r5w00000V42cCAAR,2017-10-21
4,2018-01-21,18.0,1.0,a0w5w000009Q7CyAAK,a0r5w00000V42cCAAR,2018-01-15


In [7]:
## pull the list of contacts within each cluster
contacts_in_cluster_df = db_1.query_data('Select Contact__c, Tool_Usage_Cluster__c from ContactToolClusterAssociation__c')

[Success] Bulk job creation successful. Job ID = 7505w00000TmbquAAB


{"id":"7505w00000TmbquAAB","operation":"query","object":"ContactToolClusterAssociation__c","createdById":"0055w00000DM5bOAAT","createdDate":"2021-01-22T17:11:39.000+0000","systemModstamp":"2021-01-22T17:11:39.000+0000","state":"UploadComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","retries":0,"totalProcessingTime":0}


{"id":"7505w00000TmbquAAB","operation":"query","object":"ContactToolClusterAssociation__c","createdById":"0055w00000DM5bOAAT","createdDate":"2021-01-22T17:11:39.000+0000","systemModstamp":"2021-01-22T17:11:46.000+0000","state":"JobComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":92474,"retries":0,"totalProcessingTime":4416}
[Success] Bulk job completed successfully.


In [8]:
display(contacts_in_cluster_df.head(5))

,Contact__c,Tool_Usage_Cluster__c
0,0035w000034I3dJAAS,a0w5w000009Q7ufAAC
1,0035w000034I3giAAC,a0w5w000009Q7ufAAC
2,0035w000034I3fOAAS,a0w5w000009Q7ufAAC
3,0035w000031VzX2AAK,a0w5w000009Q7ufAAC
4,0035w000034I3ffAAC,a0w5w000009Q7ufAAC


In [9]:
contacts_in_cluster_df.shape

(4987, 2)

In [10]:
## filter by dates, keep only most recent two years worth of data
import datetime
print(datetime.datetime.now())

2021-01-22 12:11:51.784135


In [11]:
now = datetime.datetime.now() 
date_diff = datetime.timedelta(days=365*2)
earliest = now-date_diff

In [12]:
earliest_string = '{year}-{month}-{day}'.format(year=earliest.year, month = earliest.month, day = earliest.day)
earliest_date = datetime.datetime.strptime(earliest_string,'%Y-%m-%d')

In [13]:
row_indexes = []
starting = tool_cluster_df['Starting_Date__c'].to_list()
#ending = tool_cluster_df['Ending_Date__c'].to_list()

#check starting dates 2 years
for i,j in enumerate(starting):
    j = datetime.datetime.strptime(j,'%Y-%m-%d')
    
    if j >= earliest:
        row_indexes.append(i)

print(len(row_indexes))

954


In [14]:
tool_cluster_df2 = tool_cluster_df.iloc[row_indexes,:].reset_index().drop(columns='index')
display(tool_cluster_df2.head(5))

,Ending_Date__c,Estimated_cluster_size__c,Estimated_number_of_tools__c,Id,Organization__c,Starting_Date__c
0,2019-11-28,39.0,3.0,a0w5w000009Q7CwAAK,a0r5w00000V42iyAAB,2019-11-04
1,2019-11-15,42.0,3.0,a0w5w000009Q7D4AAK,a0r5w00000V42cCAAR,2019-08-19
2,2019-02-24,9.0,1.0,a0w5w000009Q7ECAA0,a0r5w00000V43WSAAZ,2019-02-20
3,2019-08-20,12.0,2.0,a0w5w000009Q7EDAA0,a0r5w00000V42yLAAR,2019-08-03
4,2020-01-29,16.0,6.0,a0w5w000009Q7EEAA0,a0r5w00000V42c1AAB,2019-10-05


In [15]:
interested_ids = tool_cluster_df2['Id'].to_list()

In [16]:
## filter contacts_in_cluster_df to keep only those contacts with a valid cluster ID
cluster_indexes = []
for i,j in enumerate(contacts_in_cluster_df['Tool_Usage_Cluster__c'].to_list()):
    if j in interested_ids:
        cluster_indexes.append(i)
print(len(cluster_indexes))

496


In [17]:
contacts_in_cluster_df2 = contacts_in_cluster_df.iloc[cluster_indexes,:].reset_index().drop(columns='index')
display(contacts_in_cluster_df2.head(2))

,Contact__c,Tool_Usage_Cluster__c
0,0035w000034JwAVAA0,a0w5w00000A8NYdAAN
1,0035w00003ECPbqAAH,a0w5w00000A8NYdAAN


In [18]:
# pull contacts from SF to obtain their nanohub usernames
#earliest_string_sf = '{year}-{month}-{day}'.format(year=earliest.year, month = earliest.month, day = earliest.day)
contacts_df = db_1.query_data('Select Id, nanoHUB_username__c,nanoHUB_last_active_date__c from Contact') #\
    #where nanoHUB_last_active_date__c >='+earliest_string_sf)

[Success] Bulk job creation successful. Job ID = 7505w00000TmbrYAAR


{"id":"7505w00000TmbrYAAR","operation":"query","object":"Contact","createdById":"0055w00000DM5bOAAT","createdDate":"2021-01-22T17:11:53.000+0000","systemModstamp":"2021-01-22T17:11:53.000+0000","state":"UploadComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","retries":0,"totalProcessingTime":0}


{"id":"7505w00000TmbrYAAR","operation":"query","object":"Contact","createdById":"0055w00000DM5bOAAT","createdDate":"2021-01-22T17:11:53.000+0000","systemModstamp":"2021-01-22T17:11:53.000+0000","state":"InProgress","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":0,"retries":0,"totalProcessingTime":0}


{"id":"7505w00000TmbrYAAR","operation":"query","object":"Contact","createdById":"0055w00000DM5bOAAT","createdDate":"2021-01-22T17:11:53.000+0000","systemModstamp":"2021-01-22T17:11:53.000+0000","state":"InProgress","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":0,"retries":0,"totalProcessingTime":0}


{"id":"7505w00000TmbrYAAR","operation":"query","object":"Contact","createdById":"0055w00000DM5bOAAT","createdDate":"2021-01-22T17:11:53.000+0000","systemModstamp":"2021-01-22T17:12:19.000+0000","state":"JobComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":236479,"retries":0,"totalProcessingTime":16953}
[Success] Bulk job completed successfully.


In [19]:
contacts_df = contacts_df.dropna()
print(contacts_df.shape)
print(contacts_df.head)

(229720, 3)
<bound method NDFrame.head of                         Id nanoHUB_last_active_date__c nanoHUB_username__c
0       0035w000031Vsp1AAC                  2014-11-13             hubrepo
1       0035w000031Vsp2AAC                  2008-11-19             support
2       0035w000031Vsp3AAC                  2020-02-14            gridstat
3       0035w000031Vsp4AAC                  2008-11-11                 ncn
4       0035w000031Vsp5AAC                  2014-06-26             nanohub
...                    ...                         ...                 ...
236474  0035w00003KsD0DAAV                  2021-01-22             kim77ky
236475  0035w00003KsD0EAAV                  2021-01-22       reddykusuma33
236476  0035w00003KsD0FAAV                  2021-01-22         2017ceb1016
236477  0035w00003KsD0GAAV                  2021-01-22    ayushisharma0825
236478  0035w00003KsD0HAAV                  2021-01-22       celebritynews

[229720 rows x 3 columns]>


In [20]:
contact_sf_indexes =[]
for i,j in enumerate(contacts_df['nanoHUB_last_active_date__c'].to_list()):
    if datetime.datetime.strptime(j,'%Y-%m-%d') >= earliest_date:
        contact_sf_indexes.append(i)

In [21]:
len(contact_sf_indexes)

61357

In [22]:
contacts_df2 = contacts_df.iloc[contact_sf_indexes,:].reset_index().drop(columns='index')
display(contacts_df2.head(2))

,Id,nanoHUB_last_active_date__c,nanoHUB_username__c
0,0035w000031Vsp3AAC,2020-02-14,gridstat
1,0035w000031Vt9BAAS,2020-12-08,sheelan


## algorithm
    1. for each week, pull toolstart dataframe from db2
    2. from usernames get ids
    3. from ids see if they belong to some cluster
    4. if yes, record (increment) the simulation use and the person as a novel individual

In [23]:
## filter the starting and ending dates into weeks
week = datetime.timedelta(days=7) #a week

## build dataframe from earliest date all the way to the end
## the end is whatever is left + the previous week
ending_week = (365*2)%7 + 7 #timedelta this
ending_week_start = now-datetime.timedelta(days=ending_week)
print(ending_week_start)

2021-01-13 12:11:51.840904


In [24]:
week_construct = int(365*2/7)+1

# build dataframe
weekly_df = pd.DataFrame(columns=['Week Start','Number_Unique_Institutions','Number_Unique_Users',\
                                  'Number_Unique_Classes','Number_Cluster_Simulations','Total_Simulations_All'])

In [25]:
# build week start
week_start = []

temp = earliest_date+week
for i,j in enumerate(range(week_construct)):
    if i == 0:
        temp = earliest_date+week
    else:
        temp += week
    temp_string = '{year}-{month}-{day}'.format(year=temp.year, month = temp.month, day = temp.day)
    week_start.append(temp_string)    
    
print(week_start[:2])
print(week_start[-2:])

['2019-1-30', '2019-2-6']
['2021-1-20', '2021-1-27']


In [26]:
weekly_df['Week Start'] = week_start
weekly_df['Number_Unique_Institutions'] = 0
weekly_df['Number_Unique_Users'] = 0
weekly_df['Number_Unique_Classes'] = 0
weekly_df['Number_Cluster_Simulations'] = 0
display(weekly_df.head(2))

,Week Start,Number_Unique_Institutions,Number_Unique_Users,Number_Unique_Classes,Number_Cluster_Simulations,Total_Simulations_All
0,2019-1-30,0,0,0,0,NaN
1,2019-2-6,0,0,0,0,NaN


In [27]:
# use the filter
new_df = tool_cluster_df.iloc[row_indexes,:].reset_index().drop(columns='index')
display(new_df.head(2))

,Ending_Date__c,Estimated_cluster_size__c,Estimated_number_of_tools__c,Id,Organization__c,Starting_Date__c
0,2019-11-28,39.0,3.0,a0w5w000009Q7CwAAK,a0r5w00000V42iyAAB,2019-11-04
1,2019-11-15,42.0,3.0,a0w5w000009Q7D4AAK,a0r5w00000V42cCAAR,2019-08-19


In [28]:
ws_dt = datetime.datetime.strptime('2019-11-28','%Y-%m-%d')

ws_dt1 = datetime.datetime.strptime('2019-10-28','%Y-%m-%d')
ws_dt2 = datetime.datetime.strptime('2019-12-28','%Y-%m-%d')

In [29]:
# search 
clusters_in_week = {}
for i,j in enumerate(week_start):
    clusters_in_week_inner = []
    
    # convert to datetime
    ws_dt = datetime.datetime.strptime(j,'%Y-%m-%d')
    
    org_list = [] #to keep track of unique orgs
    
    # iterate through dataframe
    for k,h in enumerate(new_df['Starting_Date__c'].to_list()): 
        # case 1 ()[], case 2 []()
        end_date_dt = datetime.datetime.strptime(new_df.iloc[k,0],'%Y-%m-%d')
        start_date_dt = datetime.datetime.strptime(h,'%Y-%m-%d')
        
        if end_date_dt >= ws_dt and start_date_dt <= (ws_dt+week):
        # demorgan's law end_date_dt < ws_dt or start_date_dt > (ws_dt+week): 
            # add information to weekly_df
            #weekly_df.iloc[i,2] += new_df.iloc[k,1] ## this guy double counted
            weekly_df.iloc[i,3] += 1
        
            org_list.append(new_df.iloc[k,-2])
            
            clusters_in_week_inner.append(new_df.iloc[k,-3])
            
    org_list = set(org_list)
    weekly_df.iloc[i,1] = len(org_list)
    clusters_in_week[i] = clusters_in_week_inner

In [30]:
display(weekly_df.head(5))

,Week Start,Number_Unique_Institutions,Number_Unique_Users,Number_Unique_Classes,Number_Cluster_Simulations,Total_Simulations_All
0,2019-1-30,13,0,18,0,NaN
1,2019-2-6,16,0,24,0,NaN
2,2019-2-13,20,0,27,0,NaN
3,2019-2-20,21,0,28,0,NaN
4,2019-2-27,18,0,28,0,NaN


### populate number of unique users + number simulations

In [31]:
# connect with DB2
import sqlalchemy as sql

engine = sql.create_engine('mysql+pymysql://%s:%s@127.0.0.1/nanohub_metrics' \
                                               %(sql_login_params['username'], sql_login_params['password']))

In [32]:
import re #regex pattern match time
from copy import deepcopy
import time

In [33]:
start_seconds = time.time()

In [34]:
# for each week run a separate sql query
for i,j in enumerate(week_start[:1]):#[:-1]):
    #print(j)
    #print(week_start[i+1])
    sql_query = "select user, tool from toolstart where datetime between '{init_date}' and '{end_date}';".format(init_date=j,end_date = week_start[i+1])

    toolstart_df = pd.read_sql_query(sql_query, engine)
    
    ## filter for empty strings and nans
    toolstart_df['user'].replace('', pd.np.nan, inplace=True)
    toolstart_df2 = toolstart_df.dropna()
    #display(toolstart_df2.head(2))
    
    ## build set of unique usernames, and get their SF IDs
    toolstart_df_users = []
    for k,kk in enumerate(toolstart_df2['user'].to_list()): #this is a botleneck, crosscheck massive lists
        if k not in toolstart_df_users:
            toolstart_df_users.append(kk)
    #print(len(toolstart_df_users))
    
    master_sf_username_list = contacts_df['nanoHUB_username__c'].to_list()
    master_sf_id_list = contacts_df['Id'].to_list()
    
    ## pull SF ids
    toolstart_df_user_ids = [] # one-to-one inplace matching to toolstart_df_users

    desired_toolstart_users = []
    desired_toolstart_ids = []

    contacts_in_clusters = contacts_in_cluster_df2['Contact__c'].to_list()
    for k,kk in enumerate(toolstart_df_users): 
        #toolstart_df_user_ids.append(master_sf_id_list[master_sf_username_list.index(j)])  

        ## check which ids are in clusters
        try:
            if master_sf_id_list[master_sf_username_list.index(kk)] in contacts_in_clusters:
                desired_toolstart_users.append(kk)    
                desired_toolstart_ids.append(master_sf_id_list[master_sf_username_list.index(kk)])
        except:
            garb = 1 #print('user not in SF')

    ## was their class at this time? not necessarily [they may not have been in a class/cluster]
    ## build a filter for these users
    filtered_desired_toolstart_ids = []
    filtered_desired_toolstart_users = []

    weeks_user_directory = ['init']
    for h,hh in enumerate(clusters_in_week[i]): #{i:[]} where i = week no, [] clusters active for that week
        temp = [contacts_in_clusters[m] for m,mm in enumerate(contacts_in_cluster_df2['Tool_Usage_Cluster__c'])\
               if mm == hh]

        weeks_user_directory += (temp)

    if weeks_user_directory[0] == 'init':
        del weeks_user_directory[0]
    
    for k,kk in enumerate(desired_toolstart_ids): #cross check all toolstart ids with week's cluster active ids
        if kk in weeks_user_directory:
            filtered_desired_toolstart_users.append(master_sf_username_list[master_sf_id_list.index(kk)])
            filtered_desired_toolstart_ids.append(kk)
            
    
    ## pick out the total simulation usage among these filtered_desired users
    # sum up instances of toolstart_df2 with said username
    sim_sum = 0
    sim_use = toolstart_df2['user'] #each occurence of name indicates one novel use
    sim_use_string = ''.join(sim_use)
    
    ## filtered_desired_toolstart_users contains duplicate people
    filtered_desired_toolstart_users2 = set(filtered_desired_toolstart_users)
    
    for k,kk in enumerate(filtered_desired_toolstart_users2):
        match = re.findall(kk,sim_use_string)
        sim_sum += len(match)
    
    
    ## populate weekly_df
    weekly_df.iloc[i,2] = len(set(filtered_desired_toolstart_users)) #users active within clusters
    weekly_df.iloc[i,-2] = deepcopy(sim_sum) #sim usage
    weekly_df.iloc[i,-1] = len(toolstart_df_users)
    

/home/users/wang2506/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  # Remove the CWD from sys.path while we load stuff.


In [35]:
end_seconds = time.time()
diff = start_seconds-end_seconds
print(diff)

-353.80388259887695


In [36]:
import os
cwd = os.getcwd()

In [37]:
weekly_df.to_csv(cwd+'/scorecard.csv')